<center><h1> Introduction </h1></center>


* **Competition** - Given a set of images of Cassava leaves we need to classify them as one of the four diseases or healthy.
* **Data** - A collection of 21397 labelled images
* **Evaluation** - Classification accuracy

We will take a quick look at the data files and train our model in this notebook. The model can be saved and used for inference in a later notebook.

<center><h1> Importing necessary libraries </h1></center> 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
ROOT_DIR = '../input/cassava-leaf-disease-classification/'
os.listdir(ROOT_DIR)

import json # to read in the 'label_num_to_disease_map.json' file

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import cv2
import random

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB3, EfficientNetB5
from tensorflow.keras.utils import plot_model

In [ ]:
# set the training and test directory paths
TRAIN_DIR = '../input/cassava-leaf-disease-classification/train_images/'
TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'

In [ ]:
# set seed
seed = 42

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(seed)

<center><h1> Data exploration </h1></center> 

In [ ]:
train_df = pd.read_csv(ROOT_DIR + 'train.csv')
sample_df = pd.read_csv(ROOT_DIR + 'sample_submission.csv')

In [ ]:
print(train_df.shape, sample_df.shape)
display(train_df.head())

In [ ]:
f = open(ROOT_DIR + 'label_num_to_disease_map.json')
data = json.load(f)
print(json.dumps(data, indent = 2))

In [ ]:
z = train_df.sample(20)
display(z)
images, labels = z['image_id'].tolist(), z['label'].tolist()

<center><h1> Plotting 20 randomly sampled images with class </h1></center> 

In [ ]:
plt.figure(figsize = (20,20))
for i in range(20):
    plt.subplot(4,5,i+1)
    img = cv2.imread(TRAIN_DIR + images[i])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.title(data[str(labels[i])])

<center><h1> Plotting a count plot and pie chart for class distribution. </h1></center> 

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x = 'label', data = train_df, palette = 'Pastel1');

In [ ]:
pie_df = train_df['label'].value_counts().reset_index()
pie_df.columns = ['label', 'count']
fig = px.pie(pie_df, values = 'count', names = 'label', color_discrete_sequence = px.colors.qualitative.Pastel)
fig.show()

There is a class imbalance problem here. The Cassava Mosaic Disease (CMD) samples are more compared to other classes. We can solve this by either upsampling other class samples or downsampling the CMD samples.

<center><h1> Split dataset for training and validation </h1></center> 
<center> Reserving 15% of data for validation </center>

In [ ]:
train_df = train_df.astype({"label": str})

In [ ]:
train, test = train_test_split(train_df, test_size = 0.15, random_state = seed)
print(train.shape, test.shape)

<center><h1> Creating ImageDataGenerator to generate data in batches and perform image augmentation. </h1></center> 

In [ ]:
IMG_SIZE = 300
size = (IMG_SIZE,IMG_SIZE)
batch_size = 32

In [ ]:
datagen = ImageDataGenerator(
                    rotation_range = 30,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    brightness_range = [0.5,1.5],
                    horizontal_flip = True,
                    vertical_flip = True,
                    fill_mode = 'nearest'
)

In [ ]:
validgen = ImageDataGenerator()

In [ ]:
train_generator = datagen.flow_from_dataframe(
                    train,
                    directory = TRAIN_DIR,
                    x_col = "image_id",
                    y_col = "label",
                    target_size = size,
                    class_mode = "sparse",
                    batch_size = batch_size,
                    shuffle = True,
                    seed = seed,
                    interpolation = "nearest"
)

In [ ]:
valid_generator = validgen.flow_from_dataframe(
                    test,
                    directory = TRAIN_DIR,
                    x_col = "image_id",
                    y_col = "label",
                    target_size = size,
                    class_mode = "sparse",
                    batch_size = batch_size,
                    shuffle = False,
                    seed = seed,
                    interpolation = "nearest"
)

<center><h2> Visualizing some of the augmented images </h2></center>

### Image before Augmentation

In [ ]:
img = cv2.imread(os.path.join(TRAIN_DIR,'1000201771.jpg'))
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)

### Images after Augmentation

In [ ]:
plt.figure(figsize = (10,10))
# set the title
plt.title('Augmented Images')
# load the image
img = load_img(os.path.join(TRAIN_DIR,'1000201771.jpg'))
# convert to numpy array
data = img_to_array(img)
# expand dimension to one sample
samples = np.expand_dims(data, 0)
# iterator
itr = datagen.flow(samples, batch_size = 1)
# generate samples and plot
for i in range(12):
    # define subplot
    plt.subplot(4,3,i+1)
    # generate batch of images
    batch = itr.next()
    # convert to unsigned integers for viewing
    image = batch[0].astype('uint8')
    # plot raw pixel data
    plt.imshow(image)
# show the figure
plt.show()

<center><h1> Model creation and training </h1></center> 

In [ ]:
NUM_CLASSES = 5

In [ ]:
def create_model():
    
    model = models.Sequential()
    # initialize EfficientNetB3 model with input shape as (300,300,3)
    model.add(EfficientNetB3(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet'))
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256, activation = 'relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(NUM_CLASSES, activation = 'softmax'))
    
    return model

In [ ]:
model = create_model()

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = Adam(learning_rate = 0.001),
             metrics = ['accuracy'])

In [ ]:
# Stop training when the validation loss metric has stopped decreasing for 5 epochs.
early_stopping = EarlyStopping(monitor = 'val_loss',
                               patience = 5,
                               mode = 'min',
                               restore_best_weights = True)

# Save the model with the maximum validation accuracy 
checkpoint = ModelCheckpoint('best_model.hdf5', 
                             monitor = 'val_accuracy',
                             verbose = 1,
                             mode = 'max', 
                             save_best_only = True)
# reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 2,
                              mode = 'min',
                              verbose = 1)

In [ ]:
EPOCHS = 15
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size

In [ ]:
history = model.fit(train_generator,
                    validation_data = valid_generator,
                    epochs = EPOCHS,
                    steps_per_epoch = STEP_SIZE_TRAIN,
                    validation_steps = STEP_SIZE_VALID,
                    callbacks = [early_stopping, checkpoint, reduce_lr]
                   )

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes = True)

<center><h1> Model evaluation </h1></center> 

In [ ]:
model.evaluate_generator(generator = valid_generator, steps = STEP_SIZE_VALID)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'c-', label='Training accuracy')
plt.plot(epochs, val_acc, 'y-', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'c-', label='Training Loss')
plt.plot(epochs, val_loss, 'y-', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### Short version history:
* Baseline model - 0.601
* EfficientNetB0 - image_size = (224,224), batch_size =	64	- 0.841
* EfficientNetB3 - image_size =	(300,300), batch_size =	32	- 0.856
* EfficientNetB3 with more augmentations - 0.865


| Base model	  |resolution |
|-----------------|-----------|
| EfficientNetB0  |	224 |
| EfficientNetB1  |	240 |
| EfficientNetB2  |	260 |
| EfficientNetB3  |	300 |
| EfficientNetB4  |	380 |
| EfficientNetB5  |	456 |
| EfficientNetB6  |	528 |
| EfficientNetB7  |	600 |

### Some useful links:
* The prediction and submission notebook can be found here : [Inference Notebook](https://www.kaggle.com/lavanyask/cassava-leaf-disease-inference)
* More about keras EfficientNets: [here](https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/)

### Further experiments:
* Trying out different network architecture, changing number of layers in the network
* Hyperparameter tuning - changing epochs, batch size, number of neurons in hiddden layers, activation function ...
* Cross Validation
* More augmentation techniques
* PyTorch

## Do consider upvoting if you found it useful :)
### Thank you for reading the notebook.